In [1]:
#%cd C:/Users/Mathieu/Desktop/Projets/Benter
%cd /home/mathieu/Mindsay/mathieu/Benter-Project

/home/mathieu/Mindsay/mathieu/Benter-Project


In [2]:
%matplotlib inline
import collections
import functools
import json

from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
import seaborn as sns
import pandas as pd
from tqdm import tqdm
from joblib import Memory
from scipy.spatial import distance

from utils import import_data
from utils import wagering_stategy
from utils import permutations
from utils import preprocess
from winning_horse_models.logistic_regression import LogisticRegressionModel
from constants import PMU_BETTINGS, SOURCE_PMU
from utils.wagering_stategy import compute_expected_return, compute_scenario, plot_scenario
import wagering_stategies
from wagering_stategies import factories as wagering_factories
from constants import CACHE_DIR
from winning_validation.r_squared import compute_mcfadden_r_squared
from winning_validation import errors
from training_procedures import sequential_training
from typing import List, Tuple
from winning_horse_models import AbstractWinningModel
from sklearn.feature_selection import SelectKBest, mutual_info_classif, SelectFpr, f_classif

from matplotlib import style
style.use('ggplot')

winning_model = LogisticRegressionModel.load_model()
SOURCE='PMU'
source=SOURCE

In [3]:
import featuretools as ft

In [4]:
train_races_df=import_data.get_split_date(source=SOURCE, on_split="train")

/home/mathieu/.pyenv/versions/3.7.4/envs/venv_benter/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (35,46,47,61,62,63,64,87) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


In [5]:
train_races_df.columns

Index(['horse_name', 'horse_number', 'horse_age', 'horse_sex', 'horse_race',
       'statut', 'placeCorde', 'owner_name', 'trainer_name', 'jockey_name',
       'driverChange', 'tauxReclamation', 'indicateurInedit', 'musique',
       'n_run_races', 'n_won_races', 'n_placed_races', 'father_horse_name',
       'mother_horse_name', 'horse_place', 'is_pregnant', 'engagement',
       'supplement', 'handicap_distance', 'poidsConditionMonteChange',
       'horse_race_duration', 'reductionKilometrique', 'allure', 'n_reunion',
       'n_course', 'date', 'race_datetime', 'in_incident', 'incident_type',
       'totalEnjeu', 'last_race_date', 'handicap_weight',
       'is_true_total_enjeu', 'reunion_nature', 'reunion_audience',
       'reunion_pays', 'course_statut', 'course_discipline',
       'course_specialite', 'course_condition_sexe', 'course_condition_age',
       'course_track_type', 'course_penetrometre', 'course_corde',
       'course_hippodrome', 'course_parcours', 'course_distance',
    

In [ ]:
['incident', 'horse_number', 'horse_age', 'horse_sex',
 
       'statut', 'placeCorde', 'driverChange', 'tauxReclamation', 'indicateurInedit', 'musique',
       'n_run_races', 'n_won_races', 'n_placed_races', 'horse_place', 'is_pregnant', 'engagement',
       'supplement', 'handicap_distance', 'poidsConditionMonteChange',
       'horse_race_duration', 'reductionKilometrique', 'in_incident', 'incident_type',
       'totalEnjeu', 'last_race_date', 'handicap_weight',
       'is_true_total_enjeu',
 'handicapPoids',
        'poidsConditionMonte', 'blinkers',
       'unshod', 'horse_career_prize',
       'gainsVictoires', 'gainsPlace', 'horse_current_year_prize',
       'horse_last_year_prize', 'nombrePlacesSecond', 'nombrePlacesTroisieme',
       'handicap_value', 'pari_mutuel_proba',
       'odds']

horse_in_race
race_result

In [8]:
racetrack_columns = ["course_hippodrome", "reunion_pays"]
racetrack_df = (
    train_races_df[racetrack_columns]
    .drop_duplicates()
    .reset_index()
    .drop(columns="index")
    .reset_index()
)
racetrack_df.rename(
    columns={
        "course_hippodrome": "racetrack_name",
        "reunion_pays": "country",
        "index": "id",
    },
    inplace=True,
)
racetrack_df = racetrack_df[pd.notna(racetrack_df["racetrack_name"])]
assert racetrack_df["racetrack_name"].value_counts().max() == 1

horse_show_columns = ["reunion_nature", "reunion_audience", "course_hippodrome"]
horse_show_df = (
    train_races_df.groupby(["date", "n_reunion"])[horse_show_columns]
    .first()
    .reset_index()
    .reset_index()
)
horse_show_df.rename(columns={"index": "id"}, inplace=True)
horse_show_df["racetrack_id"] = horse_show_df["course_hippodrome"].map(
    racetrack_df.set_index("racetrack_name")["id"]
)
horse_show_df["racetrack_id"].fillna(0, inplace=True)
horse_show_df["racetrack_id"] = horse_show_df["racetrack_id"].astype(np.int64)

race_columns = [
    "n_course",
    "n_horses",
    "course_statut",
    "course_discipline",
    "course_specialite",
    "course_condition_sexe",
    "course_condition_age",
    "course_track_type",
    "course_penetrometre",
    "course_corde",
    "course_hippodrome",
    "course_parcours",
    "course_distance",
    "course_distance_unit",
    "course_duration",
    "course_prize_pool",
    "course_winner_prize",
    "date",
    #"race_datetime",
    "n_reunion",
    "allure",
]

race_df = train_races_df[race_columns].drop_duplicates()
race_df["name"] = race_df["date"].str.cat(
    ["R" + race_df["n_reunion"].astype(str), "C" + race_df["n_course"].astype(str)],
    sep="_",
)

assert race_df["name"].value_counts().max() == 1

race_df = race_df.reset_index(drop=True).reset_index()

race_df.rename(
    columns={"index": "id", "race_datetime": "datetime", "allure": "pace"},
    inplace=True,
)
race_df.rename(
    columns={
        column_name: column_name.replace("course_", "")
        for column_name in race_df.columns
        if column_name.startswith("course_")
    },
    inplace=True,
)

race_df["horse_show_id"] = [
    horse_show_df[
        (horse_show_df["date"] == d) & (horse_show_df["n_reunion"] == n_r)
    ]["id"].iloc[0]
    for d, n_r in zip(race_df["date"], race_df["n_reunion"])
]

race_df["horse_show_id"] = race_df["horse_show_id"].astype(np.int64)

trainer_df = pd.Series(
    train_races_df["trainer_name"].unique(), name="trainer_name"
).reset_index()
trainer_df.rename(columns={"index": "id", "trainer_name": "trainer"}, inplace=True)

jockey_df = pd.Series(
    train_races_df["jockey_name"].unique(), name="name"
).reset_index()
jockey_df.rename(columns={"index": "id"}, inplace=True)

owner_df = pd.Series(
    train_races_df["owner_name"].unique(), name="name"
).reset_index()
owner_df.rename(columns={"index": "id"}, inplace=True)

breeder_df = pd.Series(
    train_races_df["breeder_name"].unique(), name="name"
).reset_index()
breeder_df.rename(columns={"index": "id"}, inplace=True)

stable_df = pd.Series(train_races_df["ecurie"].unique(), name="name").reset_index()
stable_df.rename(columns={"index": "id"}, inplace=True)

horse_entity_set = ft.EntitySet("HorseEntitySet")
horse_entity_set.entity_from_dataframe(
    entity_id="racetrack", dataframe=racetrack_df, index="id"
)
horse_entity_set.entity_from_dataframe(
    entity_id="horse_show", dataframe=horse_show_df, index="id"
)
horse_entity_set.entity_from_dataframe(
    entity_id="race", dataframe=race_df, index="id"
)
horse_entity_set.entity_from_dataframe(
    entity_id="jockey", dataframe=jockey_df, index="id"
)
horse_entity_set.entity_from_dataframe(
    entity_id="trainer", dataframe=trainer_df, index="id"
)
horse_entity_set.entity_from_dataframe(
    entity_id="owner", dataframe=owner_df, index="id"
)
horse_entity_set.entity_from_dataframe(
    entity_id="breeder", dataframe=breeder_df, index="id"
)
horse_entity_set.entity_from_dataframe(
    entity_id="stable", dataframe=stable_df, index="id"
)
horse_entity_set.add_relationship(
    ft.Relationship(
        horse_entity_set["racetrack"]["id"],
        horse_entity_set["horse_show"]["racetrack_id"],
    )
)
horse_entity_set.add_relationship(
    ft.Relationship(
        horse_entity_set["horse_show"]["id"],
        horse_entity_set["race"]["horse_show_id"],
    )
)

Entityset: HorseEntitySet
  Entities:
    racetrack [Rows: 320, Columns: 3]
    horse_show [Rows: 11771, Columns: 7]
    race [Rows: 80250, Columns: 23]
    jockey [Rows: 16945, Columns: 2]
    trainer [Rows: 16144, Columns: 2]
    owner [Rows: 56962, Columns: 2]
    breeder [Rows: 12474, Columns: 2]
    stable [Rows: 7, Columns: 2]
  Relationships:
    horse_show.racetrack_id -> racetrack.id
    race.horse_show_id -> horse_show.id

In [28]:
dir(horse_df[horse_df['id']==0]['father_mother_horse_name'].value_counts())

['T',
 '_AXIS_ALIASES',
 '_AXIS_IALIASES',
 '_AXIS_LEN',
 '_AXIS_NAMES',
 '_AXIS_NUMBERS',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__

In [74]:
horse_df['id'].value_counts()[horse_df['id'].value_counts()==horse_df['id'].value_counts().max()]

7055     9
6083     9
14990    9
Name: id, dtype: int64

In [76]:
horse_df[horse_df['id']==6083]

,horse_race,name,id,father_id,mother_id,father_mother_id
6900,TROTTEUR ETRANGER,SAKIMA,6083,170794.0,180386.0,NaN
21594,PUR-SANG,SAKIMA,6083,100835.0,178832.0,13152.0
76826,TROTTEUR ETRANGER,SAKIMA,6083,180386.0,170794.0,NaN
111348,PUR-SANG,SAKIMA,6083,178832.0,100835.0,13152.0
177649,TROTTEUR FRANCAIS,SAKIMA,6083,180386.0,170794.0,NaN
420517,TROTTEUR FRANCAIS,SAKIMA,6083,178765.0,193925.0,NaN
491235,TROTTEUR FRANCAIS,SAKIMA,6083,178952.0,214641.0,NaN
519616,TROTTEUR ETRANGER,SAKIMA,6083,178952.0,214641.0,NaN
688129,PUR-SANG,SAKIMA,6083,179016.0,199053.0,44157.0


In [49]:
train_races_df['horse_race'].sample(10)

650484             PUR-SANG
178793             PUR-SANG
196317             PUR-SANG
549465    TROTTEUR FRANCAIS
870182             PUR-SANG
286651    TROTTEUR FRANCAIS
203938    TROTTEUR FRANCAIS
227504    TROTTEUR FRANCAIS
603376    TROTTEUR FRANCAIS
339339    TROTTEUR FRANCAIS
Name: horse_race, dtype: object

In [41]:
train_races_df['allure'].sample(10)

23210     GALOP
888208    GALOP
987741     TROT
975325     TROT
170231     TROT
68400      TROT
160029     TROT
252204    GALOP
975319     TROT
34560     GALOP
Name: allure, dtype: object

In [43]:
train_races_df['course_specialite'].sample(10)

1003705    TROT_ATTELE
896022            PLAT
434305            PLAT
205064            PLAT
382342            PLAT
380270      TROT_MONTE
295472     TROT_ATTELE
401798     TROT_ATTELE
70890      TROT_ATTELE
794696            PLAT
Name: course_specialite, dtype: object

In [21]:
trainer_df['trainer_name'].str.split().str[-1].value_counts().head(10)

(S)          215
M.            60
JR.           52
GB)           51
R.            50
C.            36
GONZALEZ      33
MARTIN        33
RODRIGUEZ     33
A.            29
Name: trainer_name, dtype: int64

In [30]:
trainer_df[trainer_df['trainer_name'].str.contains(r' MARTIN$', na=False)]['trainer_name']

642             L.J.J. MARTIN
651                 E. MARTIN
780                 P. MARTIN
947               L.A. MARTIN
2058             A. J. MARTIN
2664              L.B. MARTIN
2961            ANDREW MARTIN
3191             ED.A. MARTIN
3284            J.J.L. MARTIN
3581           PATRICK MARTIN
3833                M. MARTIN
4326                J. MARTIN
4331              L.J. MARTIN
4894               TH. MARTIN
5082              W.J. MARTIN
5232             W. J. MARTIN
5424      CESAR ANDRES MARTIN
5736     SEBASTIAN SAN MARTIN
6762           WILLIAM MARTIN
6984              G.C. MARTIN
9263              S.P. MARTIN
9268              O.C. MARTIN
10400               G. MARTIN
10776            WEBER MARTIN
12179                S MARTIN
12330               S. MARTIN
12634              A J MARTIN
13148               O. MARTIN
13207              G C MARTIN
14216              C W MARTIN
14461              T R MARTIN
14864             PISL MARTIN
15190            BELKA MARTIN
Name: trai

In [ ]:
# Appendix 


In [197]:
a=train_races_df.groupby('horse_name')['horse_sex'].nunique()

In [198]:
a=a[a==a.max()]

In [233]:
b=train_races_df.groupby('horse_name')['ecurie'].nunique()
b[b==b.max()]

horse_name
VULCAIN DU CLOS    5
Name: ecurie, dtype: int64

In [235]:
train_races_df[train_races_df['horse_name']=="VULCAIN DU CLOS"]['ecurie']

26602     NaN
36427       A
53891       B
70253       B
82776     NaN
101592    NaN
113666    NaN
127811    NaN
185909    NaN
194186      B
200832      A
218226    NaN
316333      E
331467      C
418904    NaN
524968      D
734674      C
Name: ecurie, dtype: object

In [211]:
train_races_df[train_races_df['horse_name']=='ADEN'][['date', 'horse_age','horse_sex', 'reunion_pays', 'owner_name']]

,date,horse_age,horse_sex,reunion_pays,owner_name
10728,2013-03-16,3,MALES,FRA,Ecurie des CHARMES
14374,2013-03-23,3,MALES,FRA,Ecurie des CHARMES
25702,2013-04-16,3,MALES,FRA,Ecurie des CHARMES
40345,2013-05-18,3,MALES,FRA,Ecurie des CHARMES
112020,2013-10-04,3,FEMELLES,FRA,H.H. M.AL THANI
165357,2014-01-05,4,MALES,FRA,Ecurie des CHARMES
173038,2014-01-19,4,MALES,FRA,Ecurie des CHARMES
177279,2014-01-27,4,MALES,FRA,Ecurie des CHARMES
196714,2014-03-06,4,MALES,FRA,Ecurie des CHARMES
201418,2014-03-14,4,MALES,FRA,Ecurie des CHARMES


In [224]:
owner_df=train_races_df['owner_name'].str.upper().drop_duplicates()

SC HOE                              1
D.J. HENRY                          1
G. LOUIS                            1
TAN DR K C                          1
W.ROSCHI                            1
                                   ..
MELEJ R GABRIEL                     1
MM. GAVIN GLOVER, R. & A. JUHEL     1
J.Y. ROZE                           1
T.GEHRIG                            1
MR W H W & MRS N S REUSCH, MESSR    1
Name: owner_name, Length: 55388, dtype: int64

In [221]:
train_races_df['trainer_name']

0                 V. GOETZ
1                 H. GRIFT
2              E. AUDEBERT
3                 F. TERRY
4            D. LOCQUENEUX
                ...       
1005233          M. BEZIER
1005234          D. VALLEE
1005235        L. BARASSIN
1005236         M. LECOURT
1005237    TH. DUVALDESTIN
Name: trainer_name, Length: 1005238, dtype: object